In [1]:
import math
import numpy as np
import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe

## Fundamental concepts: delays, initial values, and feedback loops

A simple model is given by

    y = dy.signal()                         # introduce variable y
    x = z + u                               # x[k] = z[k] + u[k]
    z << dy.delay(x, initial_state = 2.0)   # z[k+1] = x[k], z[0] = 2.0

Herein, a signal variable $z$ is introduced that is combined with the system input $u$. The resulting intermediate variable $x$ is then delayed and assigned to $z$ closing the feedback loop. The delay has an initial value $z[0] = 2.0$.

Given $u[k] = 1.0$, for $k>=0$, the output sequence for $y$ is then $\{ 2, 3, 4, 5, ...  \}$.

By combination of these elements (and potentially others), more sophisticated signal processing algorithms can be implemented.


In [2]:
dy.clear()

system = dy.enter_system()

# define system inputs
u               = dy.system_input( dy.DataTypeFloat64(1), name='input1', default_value=1.0,  value_range=[0, 25], title="input #1")


y = dy.signal()                         # introduce variable y
x = y + u                               # x[k] = y[k] + u[k]
y << dy.delay(x, initial_state = 2.0)   # y[k+1] = y[k] + x[k], y[0] = 2.0


# define output(s)
dy.append_primay_ouput(y, 'output')

# generate code for Web Assembly (wasm), requires emcc (emscripten) to build
code_gen_results = dy.generate_code(template=dy.WasmRuntime(), folder="generated/delay", build=False)

compiling system simulation (level 0)... 
determining the computation order...
building execution paths...
All dependencies are resolved.
Generated code will be written to generated/delay .


In [3]:
# list the blocks in the diagram
dy.show_blocks()


-------- list of blocks --------

-----------------------------
Blocks in simulation simulation:
-----------------------------
* '1' (1)
  input signals
    - output (double [1]) type proposal: (double [1])
    - input1 (double [1])
  output signals
    - s2 (double [1]) type proposal: (double [1])
* '2' (2)
  input signals
    - s2 (double [1]) type proposal: (double [1])
  output signals
    - output (double [1]) type proposal: (double [1])

  nested subsystems


In [4]:
compiled_system = dyexe.CompiledCode(code_gen_results)
testsim = dyexe.SystemInstance(compiled_system)

In [5]:
sim_results = dyexe.run_batch_simulation(testsim, input_data={}, N=10,  output_keys=['output'] )
sim_results['output']

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

# Generated source-code

The process of code generation yields in mainly two results:

- the code itself in form of c++ classes, whereby one is the main class (stored inside code_gen_results['algorithm_sourcecode'])
- a manifest (in json format) that describes which in- and outputs are defined and their datatypes (stored inside code_gen_results['manifest'])

The main class is usually called simulation (can be changes by dy.enter_system(name='XX')). Herein, the function 

    void step(Outputs & outputs, Inputs const & inputs, int calculate_outputs, bool update_states, bool reset_states)

serves as the interface and allows to perform the following actions:

- compute the current output samples of the system given the input signals (calculate_outputs=1)
- update the states given the input signals (update_states = true)
- reset the states of the system and all subsystems (calculate_outputs = true)

The recommended calling order of an algorithm embedding the source code is (!)

1. step(): reset states
2. collect current input samples (measure, ...)
3. step(): compute the outputs
4. use the output samples (store, apply to an actuator, ...)
5. step(): update the states
6. go to 2.


In [6]:
# show the manifest

from IPython.display import JSON, Code
JSON(code_gen_results['manifest'], expanded=True)

<IPython.core.display.JSON object>

In [7]:
# show the generated source code

Code(data=code_gen_results['algorithm_sourcecode'], language='c++')

// namespace for simulation {
  // global variables

  class simulation {
    public:


    // state update
    double block_2_mem;


    // state update


    //
    // cached output values
    //

    double output__block_2;

    // API-function resetStates
    void resetStates() { // created by cpp_define_function

      block_2_mem = 2.0;
    }
    // output signals of  resetStates
    struct Outputs_resetStates{
      ;

    };
    // input signals of resetStates
    struct Inputs_resetStates{
      ;

    };
    // wrapper function for resetStates
    Outputs_resetStates resetStates__(Inputs_resetStates inputs) {
      Outputs_resetStates outputs;

      resetStates();

      return outputs;
    }
    // API-function updateStates
    void updateStates(double input1) { // created by cpp_define_function
      double s2;


      // restoring the signals output from the states 
      double &output = output__block_2;


      // calculating the block outputs in the following order s2
      // that depend on input1
      // dependencies that require a state update are  

      s2 = output + input1;

      block_2_mem = s2;

      // calculating the block outputs in the following order 
      // that depend on 
      // dependencies that require a state update are  


    }
    // output signals of  updateStates
    struct Outputs_updateStates{
      ;

    };
    // input signals of updateStates
    struct Inputs_updateStates{
      double input1;

    };
    // wrapper function for updateStates
    Outputs_updateStates updateStates__(Inputs_updateStates inputs) {
      Outputs_updateStates outputs;

      updateStates(inputs.input1);

      return outputs;
    }
    // API-function calcResults_1 to compute: output
    void calcResults_1(double &output) { // created by cpp_define_function


      // calculating the block outputs in the following order output
      // that depend on 
      // dependencies that require a state update are s2 

      output = block_2_mem;

      // saving the signals output into the states 
      output__block_2 = output;
    }
    // output signals of  calcResults_1
    struct Outputs_calcResults_1{
      double output;

    };
    // input signals of calcResults_1
    struct Inputs_calcResults_1{
      ;

    };
    // wrapper function for calcResults_1
    Outputs_calcResults_1 calcResults_1__(Inputs_calcResults_1 inputs) {
      Outputs_calcResults_1 outputs;

      calcResults_1(outputs.output);

      return outputs;
    }
    // all system inputs and outputs combined
    struct Inputs{
      double input1;

    };
    struct Outputs{
      double output;

    };
    // main step function 
    void step(Outputs & outputs, Inputs const & inputs, int calculate_outputs, bool update_states, bool reset_states) {
      if (reset_states) {
        resetStates();

      }
      if (calculate_outputs==1) {
        calcResults_1(outputs.output);

      }
      if (update_states) {
        updateStates(inputs.input1);

      }

    }
  };

// end of namespace for simulation